# Get acoustic features of disclosures via Praat

In [1]:
import glob
import math
import os
import re
import subprocess
import numpy as np
import pandas as pd
import parselmouth 

from parselmouth.praat import call

# Set working directory
os.chdir('/Users/Eleanor2/Library/CloudStorage/GoogleDrive-airfire246@gmail.com/My Drive/UCR/UCR SNL/Research Projects/SyncDisclosures/Pilot/Analysis/')

In [2]:
# Set parameters
f0min = 75 # 75 for males, 100 for females
f0max = 500 # 300 for males, 500 for females
time_step = 1.0 # 1 second

Maybe parallelize this in the future: https://medium.com/@mjschillawski/quick-and-easy-parallelization-in-python-32cb9027e490

In [10]:
# Loop through participant folders & self-disclosure recordings on Google Drive
all_acoustics = []
for subdir, dirs, files in os.walk('Data/recordings'):
    for file in files: 
        
        # Only include real participants' data and self (not partner's) disclosures
        if '/P' in os.path.join(subdir, file) and 'self' in os.path.join(subdir, file):
            
            # Convert m4a file to wav
            file_path = os.path.join(subdir, file)
            wav_file_path = file_path.replace(".m4a",".wav")
            subprocess.call(['ffmpeg', '-i', file_path, wav_file_path])
            
            # Import audio recording
            sound = parselmouth.Sound(wav_file_path)
            
            # Delete wav file to save space
            os.remove(wav_file_path)
            
            # Get participant ID
            regex = re.compile('/')
            ID = regex.split(wav_file_path)[2].replace("P","")
            
            # Get disclosure name
            regex = re.compile('_')
            disclosure = regex.split(wav_file_path)[2].replace(".wav","")
            
            print('Processing P' + ID + ', disclosure ' + disclosure)
            
            # Get audio start time and end time
            tmin = math.floor(call(sound, "Get start time"))
            tmax = math.ceil(call(sound, "Get end time"))
            
            # Get timestamps
            times = list(range(tmin, tmax))
            
            # Compute pitch
            pitch = sound.to_pitch(time_step, f0min, f0max)
            pitch_values = pitch.selected_array['frequency']
            if len(pitch_values) < tmax:
                pitch_values = np.append( # if missing time points, pad with NaN
                    pitch_values, np.zeros(tmax - len(pitch_values)) + np.nan
                )
            
            # Compute intensity
            intensity = sound.to_intensity(f0min, time_step)
            intensity_values = intensity.values[0]
            if len(intensity_values) < tmax:
                intensity_values = np.append( # if missing time points, pad with NaN
                    intensity_values, np.zeros(tmax - len(intensity_values)) + np.nan
                )
                
            # Compute HF500
            hf500 = []
            for time in range(tmin, tmax):
                sound_part = call(sound, "Extract part", time - time_step, time, "rectangular", 1, "yes")
                spectrum = call(sound_part, "To Spectrum", "yes")
                top = call(spectrum, "Get highest frequency")
                lo = call(spectrum, "Get band energy", 0, 500)
                hi = call(spectrum, "Get band energy", 500, top)
                # Convert sound pressure level from pascals to DB; set to nan if zero division occurs
                try:
                    hf500_at_time = 10 * math.log10(hi / lo)
                except ZeroDivisionError:
                    hf500_at_time = np.nan
                hf500.append(hf500_at_time)
            
            # Compute jitter
            jitter = []
            for time in range(tmin, tmax):
                sound_part = call(sound, "Extract part", time - time_step, time, "rectangular", 1, "yes")
                pointProcess = call(sound_part, "To PointProcess (periodic, cc)", f0min, f0max)
                jitter_at_time = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
                jitter.append(jitter_at_time)
                
            # Compute shimmer
            shimmer = []
            for time in range(tmin, tmax):
                sound_part = call(sound, "Extract part", time - time_step, time, "rectangular", 1, "yes")
                pointProcess = call(sound_part, "To PointProcess (periodic, cc)", f0min, f0max)
                shimmer_at_time = call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
                shimmer.append(shimmer_at_time)
                
            # Compute formants
            f1_list = []
            f2_list = []
            f3_list = []
            f4_list = []
            formants = call(sound, "To Formant (burg)", 0.0025, 5, 5000, 0.025, 50)
            for time in range(tmin, tmax):
                f1 = call(formants, "Get value at time", 1, time, 'Hertz', 'Linear')
                f2 = call(formants, "Get value at time", 2, time, 'Hertz', 'Linear')
                f3 = call(formants, "Get value at time", 3, time, 'Hertz', 'Linear')
                f4 = call(formants, "Get value at time", 4, time, 'Hertz', 'Linear')
                f1_list.append(f1)
                f2_list.append(f2)
                f3_list.append(f3)
                f4_list.append(f4)
             
            # Create data frame of accoustic features & append to master list
            accoustics_df = pd.DataFrame({
                'ID': ID,
                'disclosure': disclosure,
                'time': times, 
                'pitch': pitch_values, 
                'intensity': intensity_values,
                'hf500': hf500,
                'jitter': jitter,
                'shimmer': shimmer,
                'f1': f1_list,
                'f2': f2_list,
                'f3': f3_list,
                'f4': f4_list
                })
            all_acoustics.append(accoustics_df)

Processing P136, disclosure pos2
Processing P136, disclosure pos1
Processing P136, disclosure neu1
Processing P136, disclosure neu2
Processing P136, disclosure neg1
Processing P136, disclosure neg2
Processing P131, disclosure pos2
Processing P131, disclosure pos1
Processing P131, disclosure neu1
Processing P131, disclosure neu2
Processing P131, disclosure neg1
Processing P131, disclosure neg2
Processing P138, disclosure pos2
Processing P138, disclosure pos1
Processing P138, disclosure neu1
Processing P138, disclosure neu2
Processing P138, disclosure neg1
Processing P138, disclosure neg2
Processing P107, disclosure pos2
Processing P107, disclosure pos1
Processing P107, disclosure neu1
Processing P107, disclosure neu2
Processing P107, disclosure neg1
Processing P107, disclosure neg2
Processing P100, disclosure pos2
Processing P100, disclosure pos1
Processing P100, disclosure neu1
Processing P100, disclosure neu2
Processing P100, disclosure neg1
Processing P100, disclosure neg2
Processing

Processing P67, disclosure pos1
Processing P67, disclosure neu1
Processing P67, disclosure neu2
Processing P67, disclosure neg1
Processing P67, disclosure neg2
Processing P93, disclosure pos2
Processing P93, disclosure pos1
Processing P93, disclosure neu1
Processing P93, disclosure neu2
Processing P93, disclosure neg1
Processing P93, disclosure neg2
Processing P94, disclosure pos2
Processing P94, disclosure pos1
Processing P94, disclosure neu1
Processing P94, disclosure neu2
Processing P94, disclosure neg1
Processing P94, disclosure neg2
Processing P69, disclosure pos2
Processing P69, disclosure pos1
Processing P69, disclosure neu1
Processing P69, disclosure neu2
Processing P69, disclosure neg1
Processing P69, disclosure neg2
Processing P56, disclosure pos2
Processing P56, disclosure pos1
Processing P56, disclosure neu1
Processing P56, disclosure neu2
Processing P56, disclosure neg1
Processing P56, disclosure neg2
Processing P51, disclosure pos2
Processing P51, disclosure pos1
Processi

Processing P167, disclosure pos1
Processing P167, disclosure neu1
Processing P167, disclosure neu2
Processing P167, disclosure neg1
Processing P167, disclosure neg2
Processing P158, disclosure pos2
Processing P158, disclosure pos1
Processing P158, disclosure neu1
Processing P158, disclosure neu2
Processing P158, disclosure neg1
Processing P158, disclosure neg2
Processing P160, disclosure pos2
Processing P160, disclosure pos1
Processing P160, disclosure neu1
Processing P160, disclosure neu2
Processing P160, disclosure neg1
Processing P160, disclosure neg2
Processing P156, disclosure pos2
Processing P156, disclosure pos1
Processing P156, disclosure neu1
Processing P156, disclosure neu2
Processing P156, disclosure neg1
Processing P156, disclosure neg2
Processing P151, disclosure pos2
Processing P151, disclosure pos1
Processing P151, disclosure neu1
Processing P151, disclosure neu2
Processing P151, disclosure neg1
Processing P151, disclosure neg2
Processing P105, disclosure pos2
Processing

Processing P31, disclosure neg1
Processing P31, disclosure neg2
Processing P07, disclosure pos2
Processing P07, disclosure pos1
Processing P07, disclosure neu1
Processing P07, disclosure neu2
Processing P07, disclosure neg1
Processing P07, disclosure neg2
Processing P38, disclosure pos2
Processing P38, disclosure pos1
Processing P38, disclosure neu1
Processing P38, disclosure neu2
Processing P38, disclosure neg1
Processing P38, disclosure neg2
Processing P54, disclosure pos2
Processing P54, disclosure pos1
Processing P54, disclosure neu1
Processing P54, disclosure neu2
Processing P54, disclosure neg1
Processing P54, disclosure neg2
Processing P53, disclosure pos2
Processing P53, disclosure pos1
Processing P53, disclosure neu1
Processing P53, disclosure neu2
Processing P53, disclosure neg1
Processing P53, disclosure neg2
Processing P65, disclosure pos2
Processing P65, disclosure pos1
Processing P65, disclosure neu1
Processing P65, disclosure neu2
Processing P65, disclosure neg1
Processi

In [11]:
# Concatenate into dataframe
all_acoustics_df = pd.concat(all_acoustics, ignore_index=True)
all_acoustics_df.head()

,ID,disclosure,time,pitch,intensity,hf500,jitter,shimmer,f1,f2,f3,f4
0,136,pos2,0,0.000000,41.346587,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,136,pos2,1,0.000000,47.699112,-26.733634,0.047076,0.181635,141.985867,1610.922671,2430.555861,3312.397301
2,136,pos2,2,0.000000,46.554955,-27.579789,0.051872,0.199222,242.956539,1331.537117,2616.102797,4067.682468
3,136,pos2,3,0.000000,44.523244,-25.851204,0.040639,0.185166,214.696129,1465.180280,2407.977019,3344.571641
4,136,pos2,4,131.660606,64.757791,-0.507906,0.026115,0.167235,488.792226,2522.153741,3162.531435,4450.129528


In [12]:
# Save as csv
all_acoustics_df.to_csv('Data/processed/moment_to_moment/acoustic_features_by_window_1s.csv', index=False)

### Test code for one individual recording

In [4]:
# Import audio recording
sound = parselmouth.Sound('Data/test.wav')

In [5]:
# Set parameters
f0min = 75 #75 for males, 100 for females
f0max = 500 # 300 for males, 500 for females
time_step = 1.0
tmin = math.floor(call(sound, "Get start time"))
tmax = math.ceil(call(sound, "Get end time"))

#### Compute pitch

In [62]:
pitch = sound.to_pitch(time_step, f0min, f0max)
pitch_values = pitch.selected_array['frequency']
len(pitch_values)

190

#### Compute intensity

In [85]:
intensity = sound.to_intensity(f0min, time_step)
intensity_values = intensity.values[0]
len(intensity_values)

190

#### Compute HF500

Code derived from praat implementation here: https://github.com/jjatria/plugin_prosodyad/blob/master/scripts/utterance_analysis.praat

In [8]:
hf500 = []
for time in range(tmin, tmax):
    sound_part = call(sound, "Extract part", time - time_step, time, "rectangular", 1, "yes")
    spectrum = call(sound_part, "To Spectrum", "yes")
    top = call(spectrum, "Get highest frequency")
    lo = call(spectrum, "Get band energy", 0, 500)
    hi = call(spectrum, "Get band energy", 500, top)
    # Convert sound pressure level from pascals to DB; set to nan if zero division occurs
    try:
        hf500_at_time = 10 * math.log10(hi / lo)
    except ZeroDivisionError:
        hf500_at_time = np.nan
    hf500.append(hf500_at_time)

#### Compute jitter (associated with rougher voices)

In [43]:
# local is most commonly used type of jitter
# looks like i'll have to use a for loop to get jitter over time...
# should check if this is kosher & what a good window size would be
# seems like betterup just used a one second window

jitter = []
for time in range(tmin, tmax):
    sound_part = call(sound, "Extract part", time - time_step, time, "rectangular", 1, "yes")
    pointProcess = call(sound_part, "To PointProcess (periodic, cc)", f0min, f0max)
    jitter_at_time = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
    jitter.append(jitter_at_time)

#len(jitter)

#### Compute shimmer

In [44]:
shimmer = []
for time in range(tmin, tmax):
    sound_part = call(sound, "Extract part", time - time_step, time, "rectangular", 1, "yes")
    pointProcess = call(sound_part, "To PointProcess (periodic, cc)", f0min, f0max)
    shimmer_at_time = call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    shimmer.append(shimmer_at_time)

#len(shimmer)

#### Compute formants

In [45]:
formants = call(sound, "To Formant (burg)", 0.0025, 5, 5000, 0.025, 50)

f1_list = []
f2_list = []
f3_list = []
f4_list = []

for time in range(tmin, tmax):
    f1 = call(formants, "Get value at time", 1, time, 'Hertz', 'Linear')
    f2 = call(formants, "Get value at time", 2, time, 'Hertz', 'Linear')
    f3 = call(formants, "Get value at time", 3, time, 'Hertz', 'Linear')
    f4 = call(formants, "Get value at time", 4, time, 'Hertz', 'Linear')
    f1_list.append(f1)
    f2_list.append(f2)
    f3_list.append(f3)
    f4_list.append(f4)

#### Save to data file

In [46]:
times = list(range(tmin, tmax))
ID='01'

In [48]:
df = pd.DataFrame({
    'ID': ID,
    'time': times, 
    'pitch': pitch_values, 
    'intensity': intensity_values,
    'jitter': jitter,
    'shimmer': shimmer,
    'f1': f1_list,
    'f2': f2_list,
    'f3': f3_list,
    'f4': f4_list
    })

In [49]:
df.to_csv('Data/accoustics.csv', index=False)

### Play with parallel code

In [ ]:
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm

In [ ]:
num_cores = multiprocessing.cpu_count()
recordings = [file for file in files for subdir, dirs, files in os.walk('Data/recordings') 
          if '/P' in os.path.join(subdir, file) and 'self' in os.path.join(subdir, file)]
inputs = tqdm(recordings)

In [ ]:
def get_accoustic_features

In [ ]:
if __name__ == "__main__":
    processed_list = Parallel(n_jobs=num_cores)(delayed(myfunction)(i,parameters) for i in inputs)
    